In [3]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
# import coffea
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, DelphesSchema
def readDelphes(name):
    events = NanoEventsFactory.from_root(
        name,
        schemaclass=DelphesSchema,
        treepath="Delphes",
    ).events()
    return events

In [1]:
import pandas as pd
import numpy as np

def inspect_event(events, event_idx=0, show_all_fields=False, max_particles=10):
    """
    快速查看第n个事件下各种粒子的动力学参数与标签参数
    
    Parameters:
    -----------
    events : awkward.Array
        包含事件数据的awkward数组
    event_idx : int, default=0
        要查看的事件索引
    show_all_fields : bool, default=False
        是否显示所有可用字段（包括非标准字段）
    max_particles : int, default=10
        每种粒子类型最多显示的数量
    """
    
    if event_idx >= len(events):
        print(f"错误：事件索引 {event_idx} 超出范围 (总事件数: {len(events)})")
        return
    
    event = events[event_idx]
    print(f"{'='*60}")
    print(f"事件 #{event_idx} 详细信息")
    print(f"{'='*60}")
    
    # 定义常见的动力学参数字段
    kinematic_fields = ['PT', 'Eta', 'Phi', 'Mass', 'E', 'Px', 'Py', 'Pz']
    
    # 定义常见的标签参数字段
    tag_fields = {
        'Electron': ['Charge', 'IsolVar', 'SumPtCharged', 'SumPtNeutral', 'SumPtChargedPU', 'SumPt'],
        'Muon': ['Charge', 'IsolVar', 'SumPtCharged', 'SumPtNeutral', 'SumPtChargedPU', 'SumPt'],
        'Photon': ['IsolVar', 'SumPtCharged', 'SumPtNeutral', 'SumPtChargedPU', 'SumPt'],
        'Jet': ['BTag', 'TauTag', 'Charge', 'EhadOverEem', 'NCharged', 'NNeutrals', 'Beta', 'BetaStar', 'MeanSqDeltaR', 'PTD'],
        'MissingET': ['MET', 'Eta', 'Phi']  # MissingET通常只有这些字段
    }
    
    particle_types = ['Electron', 'Muon', 'Photon', 'Jet', 'MissingET']
    
    for particle_type in particle_types:
        if particle_type not in event.fields:
            print(f"\n❌ {particle_type}: 不存在于此事件中")
            continue
            
        particles = event[particle_type]
        
        if particle_type == 'MissingET':
            # MissingET通常是标量，不是数组
            print(f"\n🔍 {particle_type}:")
            print(f"   数量: 1")
            
            # 显示MissingET的参数
            available_fields = particles.fields if hasattr(particles, 'fields') else []
            if len(available_fields) == 0:
                # 如果MissingET是标量结构
                try:
                    print(f"   MET: {particles}")
                except:
                    print(f"   数据: {particles}")
            else:
                for field in available_fields:
                    try:
                        value = particles[field]
                        print(f"   {field}: {value:.3f}" if isinstance(value, (int, float)) else f"   {field}: {value}")
                    except:
                        print(f"   {field}: 无法读取")
        else:
            # 其他粒子类型
            n_particles = len(particles)
            print(f"\n🔍 {particle_type}:")
            print(f"   数量: {n_particles}")
            
            if n_particles == 0:
                continue
                
            # 获取可用字段
            available_fields = particles.fields if hasattr(particles, 'fields') else []
            
            # 显示粒子信息
            display_count = min(n_particles, max_particles)
            
            for i in range(display_count):
                particle = particles[i]
                print(f"\n   --- {particle_type} #{i+1} ---")
                
                # 动力学参数
                kinematic_data = {}
                for field in kinematic_fields:
                    if field in available_fields:
                        try:
                            value = particle[field]
                            kinematic_data[field] = f"{value:.3f}" if isinstance(value, (int, float)) else str(value)
                        except:
                            kinematic_data[field] = "N/A"
                
                if kinematic_data:
                    print("   动力学参数:", end="")
                    for field, value in kinematic_data.items():
                        print(f" {field}={value}", end="")
                    print()
                
                # 标签参数
                tag_data = {}
                specific_tags = tag_fields.get(particle_type, [])
                
                for field in specific_tags:
                    if field in available_fields:
                        try:
                            value = particle[field]
                            if isinstance(value, (int, float)):
                                tag_data[field] = f"{value:.3f}" if abs(value) < 1000 else f"{value:.2e}"
                            else:
                                tag_data[field] = str(value)
                        except:
                            tag_data[field] = "N/A"
                
                if tag_data:
                    print("   标签参数:", end="")
                    for field, value in tag_data.items():
                        print(f" {field}={value}", end="")
                    print()
                
                # 如果要显示所有字段
                if show_all_fields:
                    other_fields = [f for f in available_fields if f not in kinematic_fields and f not in specific_tags]
                    if other_fields:
                        print("   其他字段:", end="")
                        for field in other_fields[:5]:  # 最多显示5个其他字段
                            try:
                                value = particle[field]
                                if isinstance(value, (int, float)):
                                    print(f" {field}={value:.3f}", end="")
                                else:
                                    print(f" {field}={value}", end="")
                            except:
                                print(f" {field}=N/A", end="")
                        if len(other_fields) > 5:
                            print(f" ... (+{len(other_fields)-5} more)", end="")
                        print()
            
            if n_particles > max_particles:
                print(f"   ... 还有 {n_particles - max_particles} 个 {particle_type} 未显示")
    
    print(f"\n{'='*60}")

def summary_event_statistics(events, max_events=5):
    """
    显示多个事件的粒子数量统计
    
    Parameters:
    -----------
    events : awkward.Array
        包含事件数据的awkward数组
    max_events : int, default=5
        要显示统计的事件数量
    """
    print(f"{'='*50}")
    print(f"事件统计摘要 (前 {min(max_events, len(events))} 个事件)")
    print(f"{'='*50}")
    
    particle_types = ['Electron', 'Muon', 'Photon', 'Jet']
    
    # 创建表格数据
    table_data = []
    
    for i in range(min(max_events, len(events))):
        event = events[i]
        row = {'Event': i}
        
        for particle_type in particle_types:
            if particle_type in event.fields:
                try:
                    count = len(event[particle_type])
                    row[particle_type] = count
                except:
                    row[particle_type] = 'N/A'
            else:
                row[particle_type] = 0
        
        # MissingET特殊处理
        if 'MissingET' in event.fields:
            try:
                met_value = event.MissingET
                if hasattr(met_value, 'MET'):
                    row['MET'] = f"{met_value.MET:.1f}"
                elif hasattr(met_value, '__len__') and len(met_value) > 0:
                    row['MET'] = f"{met_value[0]:.1f}" if hasattr(met_value[0], '__float__') else str(met_value[0])
                else:
                    row['MET'] = f"{float(met_value):.1f}"
            except:
                row['MET'] = 'N/A'
        else:
            row['MET'] = 'N/A'
            
        table_data.append(row)
    
    # 使用pandas显示表格（如果可用）
    try:
        df = pd.DataFrame(table_data)
        print(df.to_string(index=False))
    except:
        # 手动格式化表格
        headers = ['Event'] + particle_types + ['MET']
        print(f"{'Event':>5} {'Electron':>8} {'Muon':>6} {'Photon':>8} {'Jet':>6} {'MET':>8}")
        print("-" * 50)
        for row in table_data:
            print(f"{row['Event']:>5} {row.get('Electron', 0):>8} {row.get('Muon', 0):>6} {row.get('Photon', 0):>8} {row.get('Jet', 0):>6} {row.get('MET', 'N/A'):>8}")

# 示例使用函数
def demo_usage():
    """
    演示如何使用这些函数
    """
    print("""
使用示例:

# 查看第0个事件的详细信息
inspect_event(events, event_idx=0)

# 查看第5个事件，显示所有字段
inspect_event(events, event_idx=5, show_all_fields=True)

# 查看第3个事件，每种粒子最多显示5个
inspect_event(events, event_idx=3, max_particles=5)

# 显示前10个事件的统计摘要
summary_event_statistics(events, max_events=10)
    """)

demo_usage()


使用示例:

# 查看第0个事件的详细信息
inspect_event(events, event_idx=0)

# 查看第5个事件，显示所有字段
inspect_event(events, event_idx=5, show_all_fields=True)

# 查看第3个事件，每种粒子最多显示5个
inspect_event(events, event_idx=3, max_particles=5)

# 显示前10个事件的统计摘要
summary_event_statistics(events, max_events=10)
    


In [1]:
events=readDelphes("sample/btest.root")
inspect_event(events, event_idx=4)

NameError: name 'readDelphes' is not defined